### This scripts takes about 4 minutes seconds to execute

In [227]:
from tqdm.notebook import tqdm, trange
from importlib import reload
import horseracinglib
reload(horseracinglib)
from horseracinglib import *
pd.options.display.max_columns = None

# Load test data (not seen by models)

In [228]:
runners = pd.read_csv('data\\runners_test.csv', parse_dates=['meeting_date'], infer_datetime_format=True)
runners.sort_values(by=['race_id', 'runner_id'], inplace=True)
runners.head()

,runner_id,race_id,meeting_date,stall_number,gender,age,trainer_id,trainer_sr,sire_id,sire_sr,dam_id,dam_sr,finpos,win,win_perc,sp,mkt_prob,overage,adj_mkt_prob,blinkers,visor,cheekpieces,tonguetie,entire,gelding,daysLTO,position1,position2,position3,position1_1,position1_2,position1_3,position1_4,position2_1,position2_2,position2_3,position2_4,position3_1,position3_2,position3_3,position3_4
608,568961,440447,2013-01-01,3,G,10,3973,0.189024,50854,0.0,199460,0.0,1,1,0.703704,1.08,0.925926,1.15583,0.801095,0,0,0,0,0,1,15,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0
2819,720097,440447,2013-01-01,1,G,10,9509,0.053333,1213449,0.0,104420,0.0,4,0,0.000000,21.00,0.047619,1.15583,0.041199,0,0,0,0,0,1,365,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3087,738609,440447,2013-01-01,2,G,8,9243,0.079691,1217252,0.0,107710,0.0,3,0,0.000000,17.00,0.058824,1.15583,0.050893,0,0,0,0,0,1,225,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0
20189,1663897,440447,2013-01-01,4,G,4,19540,0.128226,261406,0.0,64884,0.0,2,0,0.000000,9.00,0.111111,1.15583,0.096131,0,0,0,0,0,1,14,4,4,2,0,0,0,1,0,0,0,1,0,1,0,0
30270,1783909,440447,2013-01-01,5,G,5,9013,0.011111,485373,0.0,441268,0.0,5,0,0.000000,81.00,0.012346,1.15583,0.010681,0,0,0,0,0,1,74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [229]:
runners_single_race = runners[runners.race_id == 440447]
runners_single_race

,runner_id,race_id,meeting_date,stall_number,gender,age,trainer_id,trainer_sr,sire_id,sire_sr,dam_id,dam_sr,finpos,win,win_perc,sp,mkt_prob,overage,adj_mkt_prob,blinkers,visor,cheekpieces,tonguetie,entire,gelding,daysLTO,position1,position2,position3,position1_1,position1_2,position1_3,position1_4,position2_1,position2_2,position2_3,position2_4,position3_1,position3_2,position3_3,position3_4
608,568961,440447,2013-01-01,3,G,10,3973,0.189024,50854,0.0,199460,0.0,1,1,0.703704,1.08,0.925926,1.15583,0.801095,0,0,0,0,0,1,15,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0
2819,720097,440447,2013-01-01,1,G,10,9509,0.053333,1213449,0.0,104420,0.0,4,0,0.000000,21.00,0.047619,1.15583,0.041199,0,0,0,0,0,1,365,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3087,738609,440447,2013-01-01,2,G,8,9243,0.079691,1217252,0.0,107710,0.0,3,0,0.000000,17.00,0.058824,1.15583,0.050893,0,0,0,0,0,1,225,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0
20189,1663897,440447,2013-01-01,4,G,4,19540,0.128226,261406,0.0,64884,0.0,2,0,0.000000,9.00,0.111111,1.15583,0.096131,0,0,0,0,0,1,14,4,4,2,0,0,0,1,0,0,0,1,0,1,0,0
30270,1783909,440447,2013-01-01,5,G,5,9013,0.011111,485373,0.0,441268,0.0,5,0,0.000000,81.00,0.012346,1.15583,0.010681,0,0,0,0,0,1,74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Demonstrate calculating model probabilities for single race

In [230]:
my_multinomial_logit = MultinomialLogitModel('models\\multinomial_logit_coefficients.csv', model_prefix='ML')
display(my_multinomial_logit.model_coefficients)
my_multinomial_logit.calculate_model_probabilities_for_single_race(runners_single_race)

,feature,coefficient
0,age,-0.027801
1,trainer_sr,5.300271
2,daysLTO,-0.000583
3,position1_1,0.548396
4,position1_2,0.539932
5,position1_3,0.324656
6,position1_4,-0.218123
7,position2_1,0.202110
8,position2_2,0.180563
9,position2_3,0.087636


,race_id,runner_id,stall_number,win,mod_prob
608,440447,568961,3,1,0.558904
2819,440447,720097,1,0,0.091615
3087,440447,738609,2,0,0.097166
20189,440447,1663897,4,0,0.152579
30270,440447,1783909,5,0,0.099735


In [231]:
my_random_choice = RandomChoiceModel(model_prefix='RC')
display(my_random_choice.model_coefficients)
my_random_choice.calculate_model_probabilities_for_single_race(runners_single_race)

None

,race_id,runner_id,stall_number,win,mod_prob
608,440447,568961,3,1,0.216251
2819,440447,720097,1,0,0.202945
3087,440447,738609,2,0,0.129734
20189,440447,1663897,4,0,0.286604
30270,440447,1783909,5,0,0.164465


In [232]:
my_favourite_choice = FavouriteChoiceModel(model_prefix='FC')
display(my_favourite_choice.model_coefficients)
my_favourite_choice.calculate_model_probabilities_for_single_race(runners_single_race)

None

,race_id,runner_id,stall_number,win,mod_prob
608,440447,568961,3,1,0.801095
2819,440447,720097,1,0,0.041199
3087,440447,738609,2,0,0.050893
20189,440447,1663897,4,0,0.096131
30270,440447,1783909,5,0,0.010681


# Demonstrate calculating model probabilities for multiple races

In [233]:
my_multinomial_logit.calculate_model_probabilities_for_multiple_races(runners)
my_multinomial_logit.model_probabilities.head()

,race_id,runner_id,stall_number,win,mod_prob
0,440447,568961,3,1,0.558904
1,440447,720097,1,0,0.091615
2,440447,738609,2,0,0.097166
3,440447,1663897,4,0,0.152579
4,440447,1783909,5,0,0.099735


In [234]:
my_random_choice.calculate_model_probabilities_for_multiple_races(runners)
my_random_choice.model_probabilities.head()

,race_id,runner_id,stall_number,win,mod_prob
0,440447,568961,3,1,0.316576
1,440447,720097,1,0,0.065492
2,440447,738609,2,0,0.409944
3,440447,1663897,4,0,0.148192
4,440447,1783909,5,0,0.059796


In [235]:
my_favourite_choice.calculate_model_probabilities_for_multiple_races(runners)
my_favourite_choice.model_probabilities.head()

,race_id,runner_id,stall_number,win,mod_prob
0,440447,568961,3,1,0.801095
1,440447,720097,1,0,0.041199
2,440447,738609,2,0,0.050893
3,440447,1663897,4,0,0.096131
4,440447,1783909,5,0,0.010681


# Calculate model accuracy

In [236]:
my_multinomial_logit.calculate_model_accuracy(runners)

0.23443708609271524

In [237]:
my_random_choice.calculate_model_accuracy(runners)

0.11754966887417219

In [225]:
my_favourite_choice.calculate_model_accuracy(runners)

0.3577814569536424

# Demonstrate probability model assessment

In [ ]:
my_model_assessment = ProbabilityModelAssessment([my_multinomial_logit, my_random_choice, my_favourite_choice], runners[['race_id', 'runner_id', 'stall_number', 'win', 'adj_mkt_prob']])
display(my_model_assessment.assessment)
my_model_assessment.perform_assessment()

In [ ]:
my_model_assessment.show_diagnostic_plots()

# Demonstrate calculating wagering strategy payoffs for single race

In [ ]:
def pick_hi_mod_prob(augmented_runners_single_race):
    stakes = pd.Series(np.full(len(augmented_runners_single_race), 0, np.float64), index=augmented_runners_single_race.index)
    max_mod_prob = np.max(augmented_runners_single_race.mod_prob)
    i = augmented_runners_single_race[augmented_runners_single_race.mod_prob == max_mod_prob].index[0]
    stakes[i] = 1
    return stakes

def pick_cond(augmented_runners_single_race):
    return 1 * ((augmented_runners_single_race.mod_prob > 0.15) & (augmented_runners_single_race.mod_prob / augmented_runners_single_race.adj_mkt_prob > 0.13))

In [ ]:
my_1st_strategy = WageringStrategy(my_multinomial_logit, pick_hi_mod_prob, 'PickHiModProb')
my_1st_strategy.calculate_strategy_stakes_and_payoffs_for_single_race(runners_single_race)

In [ ]:
my_2nd_strategy = WageringStrategy(my_random_choice, pick_hi_mod_prob, 'PickHiModProb')
my_2nd_strategy.calculate_strategy_stakes_and_payoffs_for_single_race(runners_single_race)

In [ ]:
my_3rd_strategy = WageringStrategy(my_favourite_choice, pick_hi_mod_prob, 'PickHiModProb')
my_3rd_strategy.calculate_strategy_stakes_and_payoffs_for_single_race(runners_single_race)

In [ ]:
my_4th_strategy = WageringStrategy(my_multinomial_logit, pick_cond, 'PickCond')
my_4th_strategy.calculate_strategy_stakes_and_payoffs_for_single_race(runners_single_race)

# Demonstrate calculating wagering strategy payoffs for multiple races

In [ ]:
my_1st_strategy.calculate_strategy_stakes_and_payoffs_for_multiple_races(runners)
my_1st_strategy.strategy_stakes_and_payoffs.head()

In [ ]:
my_2nd_strategy.calculate_strategy_stakes_and_payoffs_for_multiple_races(runners)
my_2nd_strategy.strategy_stakes_and_payoffs.head()

In [ ]:
my_3rd_strategy.calculate_strategy_stakes_and_payoffs_for_multiple_races(runners)
my_3rd_strategy.strategy_stakes_and_payoffs.head()

In [ ]:
my_4th_strategy.calculate_strategy_stakes_and_payoffs_for_multiple_races(runners)
my_4th_strategy.strategy_stakes_and_payoffs.head()

# Demonstrate wagering strategy assessment

In [ ]:
my_strategy_assessment = WageringStrategyAssessment([my_1st_strategy, my_2nd_strategy, my_3rd_strategy, my_4th_strategy], runners[['race_id', 'meeting_date', 'runner_id', 'stall_number', 'win', 'adj_mkt_prob']])
display(my_strategy_assessment.assessment)
my_strategy_assessment.perform_assessment()

In [ ]:
my_strategy_assessment.plot_cumulative_return()

In [ ]:
my_strategy_assessment.monthly_assessment['PickHiModProb(ML)']

In [ ]:
my_strategy_assessment.monthly_assessment['PickCond(ML)']